##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regularización de grafos para clasificación de documentos con grafos naturales

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_mlp_cora"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

La regularización de grafos es una técnica específica dentro del paradigma, mucho más amplio, del aprendizaje con grafos neuronales ([Bui et al., 2018](https://research.google/pubs/pub46568.pdf)). La idea central es la de entrenar modelos de redes neuronales con objetivos con grafos regularizados, aprovechando tanto los datos etiquetados como los no etiquetados.

En este tutorial, exploraremos el uso de la regularización de grafos para clasificar documentos que forman un grafo (orgánico) natural.

La receta general para crear un modelo de grafo regularizado con un aprendizaje estructurado neuronal (NSL) es la siguiente:

1. Generar datos de entrenamiento a partir del grafo de entrada y las características de la muestra. Los nodos del grafo corresponden a las muestras y las aristas, a la similitud entre pares de muestras. Los datos de entrenamiento resultantes contendrán las características vecinas, además de las características del nodo original.
2. Crear con una API secuencial, funcional o de subclase de `Keras` una red neuronal como modelo de base.
3. Encapsular el modelo de base con la clase del encapsulador **`GraphRegularization`**, provista por el marco de trabajo NSL, para crear un nuevo modelo `Keras` de grafo. El modelo nuevo incluirá tanto una pérdida por regularización de grafo, como el término de regularización en su objetivo de entrenamiento.
4. Entrenar y evaluar el modelo `Keras` de grafo.

## Preparación


Instalar el paquete de aprendizaje estructurado neuronal.

In [ ]:
!pip install --quiet neural-structured-learning

## Dependencias e importaciones

In [ ]:
import neural_structured_learning as nsl

import tensorflow as tf

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## Conjunto de datos Cora

El [conjunto de datos Cora](https://linqs.soe.ucsc.edu/data) es un grafo de citas en el que los nodos representan a las publicaciones de aprendizaje automático, y las aristas, a las citas entre pares de publicaciones. En esta clasificación de documentos la tarea se realiza con el objetivo de categorizar cada una de las publicaciones en alguna de las 7 categorías. En otras palabras, es un problema de clasificación en múltiples clases con 7 clases.

### Grafo

El grafo original está dirigido. Pero a efectos de este ejemplo, consideramos la versión no dirigida de este grafo. Entonces, si en la publicación A se cita la publicación B, también consideramos que la publicación B tiene una cita de A. A pesar de que no sea necesariamente cierto, en este ejemplo, consideramos a las citas como sustituto (proxy) para similitud, que, por lo común, es una propiedad conmutativa.

### Características

Cada publicación de la entrada, efectivamente, contiene dos características:

1. **Palabras**: una representación de bolsa de palabras multi-hot densa del texto de la publicación. El vocabulario para el conjunto de datos Cora contiene 1433 palabras únicas. Entonces, la longitud de esta característica es de 1433 y el valor en la posición 'i' es 0/1, lo que indica si la palabra 'i' del vocabulario existe en la publicación o no.

2. **Etiqueta**: un entero solo representa el ID de clase (la categoría) de la publicación.

### Descarga del conjunto de datos Cora

In [ ]:
!wget --quiet -P /tmp https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
!tar -C /tmp -xvzf /tmp/cora.tgz

### Conversión de los datos de Cora al formato de NSL

Para preprocesar el conjunto de datos Cora y convertirlo al formato requerido para el aprendizaje estructurado neuronal, ejecutaremos el script **'preprocess_cora_dataset.py'**, incluido en el repositorio de GitHub para NSL. Este script hace lo siguiente:

1. Genera características vecinas utilizando el grafo y las características del nodo original.
2. Genera divisiones entre los datos de entrenamiento y prueba que contienen instancias de `tf.train.Example`.
3. Hace que los datos de entrenamiento y prueba persistan en el formato `TFRecord`.

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/neural-structured-learning/master/neural_structured_learning/examples/preprocess/cora/preprocess_cora_dataset.py

!python preprocess_cora_dataset.py \
--input_cora_content=/tmp/cora/cora.content \
--input_cora_graph=/tmp/cora/cora.cites \
--max_nbrs=5 \
--output_train_data=/tmp/cora/train_merged_examples.tfr \
--output_test_data=/tmp/cora/test_examples.tfr

## Variables globales

Las rutas de archivos para los datos de entrenamiento y prueba se basan en los valores de indicadores (bandera) de la línea de comandos utilizados para invocar el script **'preprocess_cora_dataset.py'** que vimos arriba.

In [ ]:
### Experiment dataset
TRAIN_DATA_PATH = '/tmp/cora/train_merged_examples.tfr'
TEST_DATA_PATH = '/tmp/cora/test_examples.tfr'

### Constants used to identify neighbor features in the input.
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'

## Hiperparámetros

Utilizaremos una instancia de `HParams` para incluir varios hiperparámetros y contantes que se usan para el entrenamiento y la evaluación. A continuación, describimos brevemente cada una de ellas:

- **num_classes**: son un total de 7 clases diferentes

- **max_seq_length**: es el tamaño del vocabulario y todas las instancias de la entrada que tienen una representación de bolsa de palabras multi-hot densa. En otras palabras, un valor de 1 para una palabra indica que esa palabra está presente en la entrada y un valor de 0 indica que no lo está.

- **distance_type**: es la métrica de distancia utilizada para regularizar la muestra con sus vecinos.

- **graph_regularization_multiplier**: controla el peso relativo del término de regularización del grafo en la función de pérdida general.

- **num_neighbors**: la cantidad de vecinos utilizados para la regularización del grafo. Este valor tiene que ser menor o igual que el argumento de línea de comandos `max_nbrs` utilizado arriba cuando ejecutamos `preprocess_cora_dataset.py`.

- **num_fc_units**: la cantidad de capas totalmente conectadas de nuestra red neuronal.

- **train_epochs**: la cantidad de épocas de entrenamiento.

- **batch_size**: el tamaño del lote para entrenamiento y evaluación.

- **dropout_rate**: controla la tasa de dilución que sigue a cada capa totalmente conectada.

- **eval_steps**: la cantidad de lotes a procesar antes de que la evaluación se complete. Si se determina como `None`, se evalúan todas las instancias del conjunto de prueba.

In [ ]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### dataset parameters
    self.num_classes = 7
    self.max_seq_length = 1433
    ### neural graph learning parameters
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 1
    ### model architecture
    self.num_fc_units = [50, 50]
    ### training parameters
    self.train_epochs = 100
    self.batch_size = 128
    self.dropout_rate = 0.5
    ### eval parameters
    self.eval_steps = None  # All instances in the test set are evaluated.

HPARAMS = HParams()

## Carga de los datos de entrenamiento y prueba

Tal como se ha descripto antes en este bloc de notas, los datos de entrenamiento y prueba han sido creados por **'preprocess_cora_dataset.py'**. Los cargaremos en dos objetos `tf.data.Dataset`, uno para entrenamiento y otro para prueba.

En la capa de entrada de nuestro modelo, extraeremos no solo las características de las "palabras" y "etiquetas" de cada muestra, sino que además obtendremos las características vecinas correspondientes, basándonos en el valor `hparams.num_neighbors`. A las instancias con menos vecinas que `hparams.num_neighbors`, se les asignarán valores ficticios en los casos en los que no existan características vecinas.

In [ ]:
def make_dataset(file_path, training=False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """

  def parse_example(example_proto):
    """Extracts relevant fields from the `example_proto`.

    Args:
      example_proto: An instance of `tf.train.Example`.

    Returns:
      A pair whose first value is a dictionary containing relevant features
      and whose second value contains the ground truth label.
    """
    # The 'words' feature is a multi-hot, bag-of-words representation of the
    # original raw text. A default value is required for examples that don't
    # have the feature.
    feature_spec = {
        'words':
            tf.io.FixedLenFeature([HPARAMS.max_seq_length],
                                  tf.int64,
                                  default_value=tf.constant(
                                      0,
                                      dtype=tf.int64,
                                      shape=[HPARAMS.max_seq_length])),
        'label':
            tf.io.FixedLenFeature((), tf.int64, default_value=-1),
    }
    # We also extract corresponding neighbor features in a similar manner to
    # the features above during training.
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i,
                                         NBR_WEIGHT_SUFFIX)
        feature_spec[nbr_feature_key] = tf.io.FixedLenFeature(
            [HPARAMS.max_seq_length],
            tf.int64,
            default_value=tf.constant(
                0, dtype=tf.int64, shape=[HPARAMS.max_seq_length]))

        # We assign a default value of 0.0 for the neighbor weight so that
        # graph regularization is done on samples based on their exact number
        # of neighbors. In other words, non-existent neighbors are discounted.
        feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
            [1], tf.float32, default_value=tf.constant([0.0]))

    features = tf.io.parse_single_example(example_proto, feature_spec)

    label = features.pop('label')
    return features, label

  dataset = tf.data.TFRecordDataset([file_path])
  if training:
    dataset = dataset.shuffle(10000)
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  return dataset


train_dataset = make_dataset(TRAIN_DATA_PATH, training=True)
test_dataset = make_dataset(TEST_DATA_PATH)

Echemos un vistazo al conjunto de datos de entrenamiento para observar su contenido.

In [ ]:
for feature_batch, label_batch in train_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, 0, 'words')
  nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, 0, NBR_WEIGHT_SUFFIX)
  print('Batch of neighbor inputs:', feature_batch[nbr_feature_key])
  print('Batch of neighbor weights:',
        tf.reshape(feature_batch[nbr_weight_key], [-1]))
  print('Batch of labels:', label_batch)

Ahora, echemos un vistazo al conjunto de datos de prueba para observar su contenido.

In [ ]:
for feature_batch, label_batch in test_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  print('Batch of labels:', label_batch)

## Definición del modelo

Con el objetivo de demostrar el uso de la regularización de grafos, primero, creamos un modelo de base para este problema. Usaremos la red neuronal prealimentada simple con 2 capas ocultas y dilución entre capas. Ilustramos la creación del modelo base usando todos los tipos de modelos compatibles con el marco de trabajo `tf.Keras`, secuencial, funcional y de subclase.

### Modelo base secuencial

In [ ]:
def make_mlp_sequential_model(hparams):
  """Creates a sequential multi-layer perceptron model."""
  model = tf.keras.Sequential()
  model.add(
      tf.keras.layers.InputLayer(
          input_shape=(hparams.max_seq_length,), name='words'))
  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  model.add(
      tf.keras.layers.Lambda(lambda x: tf.keras.backend.cast(x, tf.float32)))
  for num_units in hparams.num_fc_units:
    model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    # For sequential models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    model.add(tf.keras.layers.Dropout(hparams.dropout_rate))
  model.add(tf.keras.layers.Dense(hparams.num_classes))
  return model

### Modelo base funcional

In [ ]:
def make_mlp_functional_model(hparams):
  """Creates a functional API-based multi-layer perceptron model."""
  inputs = tf.keras.Input(
      shape=(hparams.max_seq_length,), dtype='int64', name='words')

  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  cur_layer = tf.keras.layers.Lambda(
      lambda x: tf.keras.backend.cast(x, tf.float32))(
          inputs)

  for num_units in hparams.num_fc_units:
    cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
    # For functional models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    cur_layer = tf.keras.layers.Dropout(hparams.dropout_rate)(cur_layer)

  outputs = tf.keras.layers.Dense(hparams.num_classes)(cur_layer)

  model = tf.keras.Model(inputs, outputs=outputs)
  return model

### Modelo base de subclase

In [ ]:
def make_mlp_subclass_model(hparams):
  """Creates a multi-layer perceptron subclass model in Keras."""

  class MLP(tf.keras.Model):
    """Subclass model defining a multi-layer perceptron."""

    def __init__(self):
      super(MLP, self).__init__()
      # Input is already one-hot encoded in the integer format. We create a
      # layer to cast it to floating point format here.
      self.cast_to_float_layer = tf.keras.layers.Lambda(
          lambda x: tf.keras.backend.cast(x, tf.float32))
      self.dense_layers = [
          tf.keras.layers.Dense(num_units, activation='relu')
          for num_units in hparams.num_fc_units
      ]
      self.dropout_layer = tf.keras.layers.Dropout(hparams.dropout_rate)
      self.output_layer = tf.keras.layers.Dense(hparams.num_classes)

    def call(self, inputs, training=False):
      cur_layer = self.cast_to_float_layer(inputs['words'])
      for dense_layer in self.dense_layers:
        cur_layer = dense_layer(cur_layer)
        cur_layer = self.dropout_layer(cur_layer, training=training)

      outputs = self.output_layer(cur_layer)

      return outputs

  return MLP()

## Creación de modelos base

In [ ]:
# Create a base MLP model using the functional API.
# Alternatively, you can also create a sequential or subclass base model using
# the make_mlp_sequential_model() or make_mlp_subclass_model() functions
# respectively, defined above. Note that if a subclass model is used, its
# summary cannot be generated until it is built.
base_model_tag, base_model = 'FUNCTIONAL', make_mlp_functional_model(HPARAMS)
base_model.summary()

## Entrenamiento de modelo MLP base

In [ ]:
# Compile and train the base MLP model
base_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
base_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

## Evaluación de modelo MLP base

In [ ]:
# Helper function to print evaluation metrics.
def print_metrics(model_desc, eval_metrics):
  """Prints evaluation metrics.

  Args:
    model_desc: A description of the model.
    eval_metrics: A dictionary mapping metric names to corresponding values. It
      must contain the loss and accuracy metrics.
  """
  print('\n')
  print('Eval accuracy for ', model_desc, ': ', eval_metrics['accuracy'])
  print('Eval loss for ', model_desc, ': ', eval_metrics['loss'])
  if 'graph_loss' in eval_metrics:
    print('Eval graph loss for ', model_desc, ': ', eval_metrics['graph_loss'])

In [ ]:
eval_results = dict(
    zip(base_model.metrics_names,
        base_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('Base MLP model', eval_results)

## Entrenamiento del modelo MLP con regularización de grafo

Con solo unas líneas de código es posible incorporar la regularización del grafo en el término de pérdida de un `tf.Keras.Model` existente. El modelo base se encapsula para crear un nuevo modelo de subclase `tf.Keras`, cuya pérdida incluye la regularización del grafo.

Para evaluar el beneficio incremental de la regularización de grafos, crearemos una nueva instancia del modelo de base. El motivo es que el `base_model` ya ha sido entrenado para unas pocas iteraciones y reutilizarlo para crear un modelo de grafo regularizado no ofrecería un elemento de comparación verdadero para el `base_model`.

In [ ]:
# Build a new base MLP model.
base_reg_model_tag, base_reg_model = 'FUNCTIONAL', make_mlp_functional_model(
    HPARAMS)

In [ ]:
# Wrap the base MLP model with graph regularization.
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=HPARAMS.num_neighbors,
    multiplier=HPARAMS.graph_regularization_multiplier,
    distance_type=HPARAMS.distance_type,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
graph_reg_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

## Evaluación del modelo MLP con regularización de grafo

In [ ]:
eval_results = dict(
    zip(graph_reg_model.metrics_names,
        graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('MLP + graph regularization', eval_results)

La exactitud del modelo de grafo regularizado es de entre un 2 y un 3% superior a la del modelo base (`base_model`).

## Conclusión

Hemos demostrado el uso de la regularización de grafos para la clasificación de documentación en un grafo de citas naturales (Cora) con aprendizaje estructurado neuronal (NSL). En nuestro [tutorial avanzado](graph_keras_lstm_imdb.ipynb) se incluye la sintetización de grafos basada en incorporaciones de muestras antes de entrenar una red neuronal con regularización de grafos. Este enfoque resulta útil si la entrada no contiene un grafo explícito.

Les recomendamos a los usuarios seguir experimentando con la variación de la cantidad de supervisión y seguir probando diferentes arquitecturas neuronales para la regularización de grafos.